# LeNet CNN

In [1]:
import tensorflow as tf
import numpy as np

/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<img src="optimizer_graph.png" width="200px">

## Reading Data

We are reading MNIST data from http://yann.lecun.com/exdb/mnist/


In [14]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
model_path = "/Users/omerorhan/Desktop/UCSC/Deep Learning" \
             "/DeepLearningUCSC/venv/homework/Assignments/model.ckpt"

Extracting /tmp/data/train-images-idx3-ubyte.gz


Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
print(mnist.train.num_examples) # Number of training data
print(mnist.test.num_examples) # Number of test data


55000
10000


## Setting hyper-parameters

In [16]:
# architecture hyper-parameter
noofdatapoints = mnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = 10#noofdatapoints//batch_size
print("no of batches =", noofbatches)

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

no of batches = 10


## Defining x and y as placeholders

In [17]:
X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

(?, 784) (?, 10)


## Defining model using conv2d, Relu and Maxpooling

In [18]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [19]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [20]:
def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Initializing Weights and Biases

In [21]:


weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Define cost, optimizer and accuracy

In [22]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))
# Initializing the variables
init = tf.global_variables_initializer()
weights_saver = tf.train.Saver(var_list=weights)


Tensor("Add_3:0", shape=(?, 10), dtype=float32)


## Running the session 
This will run the graph and use all the tensors that were previously defined

In [23]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_y = batch_y.astype(np.float32)
            # Use training data for optimization
            sess.run(train_min, feed_dict={X:batch_x, Y:batch_y, keep_prob: dropout})
        # Validate after every epoch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy], 
                                          feed_dict={X:batch_x, Y:batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f"%(epoch, losscalc, accuracycalc))
            
    # When the training is complete and you are happy with the result
    path = weights_saver.save(sess,"deeplearning")

    accuracycalc = sess.run(accuracy, 
                            feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f"%(accuracycalc))

    print(path)


Epoch: 0, Loss: 25823.1660, Accuracy: 0.3203


Epoch: 1, Loss: 11216.0098, Accuracy: 0.5703


Epoch: 2, Loss: 5902.7280, Accuracy: 0.7188


Epoch: 3, Loss: 4243.4136, Accuracy: 0.7773


Epoch: 4, Loss: 3144.9917, Accuracy: 0.8125


...

Epoch: 6, Loss: 3115.9595, Accuracy: 0.8555


Epoch: 7, Loss: 2229.2986, Accuracy: 0.8848


Epoch: 8, Loss: 1654.6868, Accuracy: 0.9043


Epoch: 9, Loss: 2036.9158, Accuracy: 0.8945


Testing accuracy: 0.9037
deeplearning


# LeNet CNN with Fashion MNIST


In [13]:
from tensorflow.examples.tutorials.mnist import input_data
fmnist = input_data.read_data_sets('data/fashion', one_hot=True)


Extracting data/fashion/train-images-idx3-ubyte.gz


Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz


In [14]:
# architecture hyper-parameter
noofdatapoints = fmnist.train.num_examples

learningrate = 0.001
nepochs = 10
batch_size = 512
noofbatches = 10#noofdatapoints//batch_size
print("no of batches =", noofbatches)

n_input = 784 # 28x28 image
n_classes = 10 # 1 for each digit [0-9]
dropout = 0.75 

X = tf.placeholder(tf.float32, [None, n_input])
Y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # reshape input to 28x28 size
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max pooling
    conv1 = maxpool2d(conv1, k=2)

    # Convolution layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max pooling
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


no of batches = 10
(?, 784) (?, 10)


In [15]:



weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [16]:
# Create the model
model = conv_net(X, weights, biases, keep_prob)
print(model)
# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
train_min = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(loss)

# Evaluate model
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_model, tf.float32))
# Initializing the variables
init = tf.global_variables_initializer()

Tensor("Add_5:0", shape=(?, 10), dtype=float32)


In [19]:
with tf.Session() as sess:
    sess.run(init)
    weights_saver.restore(sess, "deeplearning")
    print("Model restored.")
    for epoch in range(nepochs):
        for _ in range(noofbatches):
            batch_x, batch_y = fmnist.train.next_batch(batch_size)
            sess.run(train_min, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        batch_x, batch_y = fmnist.train.next_batch(batch_size)
        losscalc, accuracycalc = sess.run([loss, accuracy],
                                          feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0})
        print("Epoch: %d, Loss: %0.4f, Accuracy: %0.4f" % (epoch, losscalc, accuracycalc))
    accuracycalc = sess.run(accuracy,
                            feed_dict={X: fmnist.test.images, Y: fmnist.test.labels, keep_prob: 1.0})
    print("Testing accuracy: %0.4f" % (accuracycalc))


INFO:tensorflow:Restoring parameters from deeplearning


Model restored.
aa
[ 1.1675711   0.95874965  0.02622175  0.33122376 -0.78005075  1.1321908
 -0.7139748  -0.9771778  -0.14711846 -0.19316591]
aa
[ 0.7127246   0.80073804  0.6937006  -0.797635    0.1472153   1.4478707
 -0.55439186  0.19284292 -0.47005078  0.1140357  -0.8952039  -0.02831665
 -0.96855855  1.7771686   1.2746913  -1.3278888  -0.3315122  -0.61794925
  0.42568308  0.6295675 ]


KeyboardInterrupt: 